# Exercise - Multi step Workflow - STARTER

In this exercise, you’ll build a multi-step workflow using LCEL to solve a more complex task than simply generating a joke. 

**Challenge**

Create an AI Business Advisor that:

1. Accepts an industry as input.
2. Generates a business idea.
3. Analyzes the strengths and weaknesses.
4. Formats the results as a final report.


## 0. Import the necessary libs

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from pydantic import BaseModel, Field

## 1. Instantiate Chat Model with your API Key

To be able to connect with OpenAI, you need to instantiate an ChatOpenAI client passing your OpenAI key.

You can pass the `api_key` argument directly.
```python
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key="voc-",
)
```

In [2]:
# TODO - Instantiate your chat model
OPENAI_API_KEY="voc-*"

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key = OPENAI_API_KEY,
)

## 2. Your first Chain

In the end of each chain, you should parse the output and save the logs

In [3]:
logs = []

In [4]:
parser = StrOutputParser()

In [5]:
parse_and_log_output_chain = RunnableParallel(
    output=parser, 
    log=RunnableLambda(lambda x: logs.append(x))
)

## 3. Idea Generation

Craft a prompt to generate a business idea for the given industry. 

Make sure {industry} placeholder is inside your template, so it can be filled when the chain is invoked.

In [6]:
# TODO - Your prompt Template
idea_prompt = PromptTemplate(
    template=(
        "Generate a business idea for the industry {industry}.\n" 
        "Make sure the business idea is financially sound with potential for great revenue generation \n"
        "and applies the technology {technology}.\n"
    )
)

In [7]:
# TODO - Create your idea_chain: idea_prompt -> llm -> parse_and_log_output_chain
idea_chain = idea_prompt | llm | parse_and_log_output_chain

In [8]:
# TODO - Test your idea_chain invoking it by passing an industy like "agro" to it
idea_result = idea_chain.invoke({"industry": "Financial Services", "technology": "quantum portfolio optimization"})

In [9]:
idea_result["output"]

"### Business Idea: Quantum Wealth Management Platform (QWMP)\n\n#### Overview:\nThe Quantum Wealth Management Platform (QWMP) is a cutting-edge financial services solution that leverages quantum computing technology to provide advanced portfolio optimization and risk management for individual and institutional investors. By utilizing quantum algorithms, QWMP aims to outperform traditional portfolio management strategies, offering clients superior returns and enhanced risk-adjusted performance.\n\n#### Key Features:\n\n1. **Quantum Portfolio Optimization**:\n   - Utilize quantum algorithms to analyze vast datasets and identify optimal asset allocations that traditional methods may overlook.\n   - Provide real-time adjustments to portfolios based on market conditions, risk tolerance, and investment goals.\n\n2. **Risk Assessment and Management**:\n   - Implement quantum simulations to model complex financial scenarios and assess potential risks more accurately.\n   - Offer clients tailo

In [10]:
logs

[AIMessage(content="### Business Idea: Quantum Wealth Management Platform (QWMP)\n\n#### Overview:\nThe Quantum Wealth Management Platform (QWMP) is a cutting-edge financial services solution that leverages quantum computing technology to provide advanced portfolio optimization and risk management for individual and institutional investors. By utilizing quantum algorithms, QWMP aims to outperform traditional portfolio management strategies, offering clients superior returns and enhanced risk-adjusted performance.\n\n#### Key Features:\n\n1. **Quantum Portfolio Optimization**:\n   - Utilize quantum algorithms to analyze vast datasets and identify optimal asset allocations that traditional methods may overlook.\n   - Provide real-time adjustments to portfolios based on market conditions, risk tolerance, and investment goals.\n\n2. **Risk Assessment and Management**:\n   - Implement quantum simulations to model complex financial scenarios and assess potential risks more accurately.\n   - 

## 4. Idea Analysis

Craft a prompt to analyze the generated idea's strengths and weaknesses

In [11]:
# TODO - Your prompt Template
analysis_prompt = PromptTemplate(
    template=(
        "Analyze the business idea {idea} and list its key strengths and weaknesses. List 5 main strengths and 5 main weaknesses based \n" 
        "on current market trends and reports on maturity of the applied technology. List each strength and weakness in JSON format \n "
        "with the strength or weakness as key and its explanation as value."
    )
)

In [12]:
# TODO - Your chain
analysis_chain = ( analysis_prompt | llm | parse_and_log_output_chain )

In [13]:
# TODO - Test your analysis_chain invoking it by passing idea_result["output"] to it
analysis_result = analysis_chain.invoke(idea_result["output"])

In [14]:
analysis_result["output"]

'Here’s an analysis of the Quantum Wealth Management Platform (QWMP) with its key strengths and weaknesses presented in JSON format:\n\n### Key Strengths and Weaknesses\n\n```json\n{\n  "strengths": {\n    "First-Mover Advantage": "QWMP can establish itself as a pioneer in integrating quantum computing into wealth management, attracting early adopters and building brand loyalty.",\n    "Enhanced Performance": "The use of quantum algorithms for portfolio optimization and risk management can lead to superior returns and risk-adjusted performance compared to traditional methods.",\n    "Scalability": "The platform\'s architecture allows for easy scaling to accommodate a growing user base and the potential to expand into new markets and regions.",\n    "Personalization": "By leveraging machine learning alongside quantum computing, QWMP can offer highly personalized investment strategies tailored to individual client profiles and preferences.",\n    "Sustainability Focus": "Incorporating ES

In [15]:
logs

[AIMessage(content="### Business Idea: Quantum Wealth Management Platform (QWMP)\n\n#### Overview:\nThe Quantum Wealth Management Platform (QWMP) is a cutting-edge financial services solution that leverages quantum computing technology to provide advanced portfolio optimization and risk management for individual and institutional investors. By utilizing quantum algorithms, QWMP aims to outperform traditional portfolio management strategies, offering clients superior returns and enhanced risk-adjusted performance.\n\n#### Key Features:\n\n1. **Quantum Portfolio Optimization**:\n   - Utilize quantum algorithms to analyze vast datasets and identify optimal asset allocations that traditional methods may overlook.\n   - Provide real-time adjustments to portfolios based on market conditions, risk tolerance, and investment goals.\n\n2. **Risk Assessment and Management**:\n   - Implement quantum simulations to model complex financial scenarios and assess potential risks more accurately.\n   - 

## 5. Report Generation

Craft a prompt to structure the information into a business report.

In [16]:
# TODO - Your prompt Template
report_prompt = PromptTemplate(
    template=(
        "Take the analysis result {output} and structure the information in form of a business report including all the key strengths and weaknesses of the analyzed business idea." 
    )
)

In [17]:
class AnalysisReport(BaseModel):
    """Strengths and Weaknesses about a business idea"""
    strengths: list = Field(default=[], description="Idea's strengths list")
    weaknesses: list = Field(default=[], description="Idea's weaknesses list")

In [18]:
report_chain = (
    report_prompt | llm.with_structured_output(schema=AnalysisReport, method="function_calling")
)

In [19]:
# TODO - Test your report_chain invoking it by passing analysis_result["output"] to it
report_result = report_chain.invoke(analysis_result["output"])

In [20]:
report_result

AnalysisReport(strengths=['First-Mover Advantage: QWMP can establish itself as a pioneer in integrating quantum computing into wealth management, attracting early adopters and building brand loyalty.', 'Enhanced Performance: The use of quantum algorithms for portfolio optimization and risk management can lead to superior returns and risk-adjusted performance compared to traditional methods.', "Scalability: The platform's architecture allows for easy scaling to accommodate a growing user base and the potential to expand into new markets and regions.", 'Personalization: By leveraging machine learning alongside quantum computing, QWMP can offer highly personalized investment strategies tailored to individual client profiles and preferences.', "Sustainability Focus: Incorporating ESG factors into investment strategies appeals to the growing segment of socially conscious investors, enhancing the platform's marketability."], weaknesses=['Technology Maturity: Quantum computing is still in its

In [21]:
report_result.strengths

['First-Mover Advantage: QWMP can establish itself as a pioneer in integrating quantum computing into wealth management, attracting early adopters and building brand loyalty.',
 'Enhanced Performance: The use of quantum algorithms for portfolio optimization and risk management can lead to superior returns and risk-adjusted performance compared to traditional methods.',
 "Scalability: The platform's architecture allows for easy scaling to accommodate a growing user base and the potential to expand into new markets and regions.",
 'Personalization: By leveraging machine learning alongside quantum computing, QWMP can offer highly personalized investment strategies tailored to individual client profiles and preferences.',
 "Sustainability Focus: Incorporating ESG factors into investment strategies appeals to the growing segment of socially conscious investors, enhancing the platform's marketability."]

In [22]:
report_result.weaknesses

['Technology Maturity: Quantum computing is still in its early stages, and the practical application of quantum algorithms in finance may face technical challenges and limitations.',
 'High Development Costs: The initial investment required for developing and maintaining quantum computing infrastructure and expertise can be substantial, impacting profitability in the early stages.',
 'Regulatory Uncertainty: The financial services industry is heavily regulated, and the introduction of new technologies like quantum computing may face scrutiny and regulatory hurdles.',
 'Market Education: There may be a steep learning curve for potential clients to understand quantum finance, necessitating significant investment in educational resources and marketing.',
 'Competition from Established Firms: Traditional wealth management firms may quickly adapt to new technologies or develop their own quantum solutions, increasing competition in the market.']

## 6. End to End Chain

In [23]:
!pip install grandalf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 5.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [24]:
import grandalf

In [25]:
e2e_chain = ( 
    RunnablePassthrough() 
    | idea_chain
    | RunnableParallel(idea=RunnablePassthrough())
    | analysis_chain
    | report_chain
)

In [26]:
e2e_chain.get_graph().print_ascii()

            +------------------+         
            | PassthroughInput |         
            +------------------+         
                      *                  
                      *                  
                      *                  
              +-------------+            
              | Passthrough |            
              +-------------+            
                      *                  
                      *                  
                      *                  
             +----------------+          
             | PromptTemplate |          
             +----------------+          
                      *                  
                      *                  
                      *                  
               +------------+            
               | ChatOpenAI |            
               +------------+            
                      *                  
                      *                  
                      *           

In [27]:
# Change the industry if you want
e2e_result = e2e_chain.invoke({"industry": "agro", "technology": "sustainable technologies"})

In [28]:
e2e_result

AnalysisReport(strengths=['High Yield Production: Vertical farms can produce significantly more crops per square foot compared to traditional farming, maximizing the use of limited urban space and increasing revenue potential.', 'Sustainability Focus: The use of hydroponics, aeroponics, and renewable energy sources aligns with growing consumer demand for sustainable and environmentally friendly practices, enhancing brand appeal.', 'Local Supply Chain: By operating in urban areas, the business can reduce transportation costs and time to market, ensuring fresher produce and lower carbon emissions, which is increasingly important to consumers.', 'Technological Integration: The incorporation of IoT and smart monitoring systems allows for data-driven decision-making, optimizing growth conditions and improving overall efficiency and productivity.', 'Diverse Revenue Streams: The business model includes multiple revenue streams such as direct sales, subscription services, and educational works

In [29]:
e2e_result.strengths

['High Yield Production: Vertical farms can produce significantly more crops per square foot compared to traditional farming, maximizing the use of limited urban space and increasing revenue potential.',
 'Sustainability Focus: The use of hydroponics, aeroponics, and renewable energy sources aligns with growing consumer demand for sustainable and environmentally friendly practices, enhancing brand appeal.',
 'Local Supply Chain: By operating in urban areas, the business can reduce transportation costs and time to market, ensuring fresher produce and lower carbon emissions, which is increasingly important to consumers.',
 'Technological Integration: The incorporation of IoT and smart monitoring systems allows for data-driven decision-making, optimizing growth conditions and improving overall efficiency and productivity.',
 'Diverse Revenue Streams: The business model includes multiple revenue streams such as direct sales, subscription services, and educational workshops, which can help 

In [30]:
e2e_result.weaknesses

['High Initial Investment: The startup costs for technology, infrastructure, and equipment can be substantial, which may pose a barrier to entry and require significant funding.',
 'Operational Complexity: Managing a vertical farm with advanced technology and sustainable practices can be complex, requiring skilled labor and ongoing training, which may increase operational costs.',
 'Market Competition: The vertical farming market is becoming increasingly competitive, with many players entering the space, which could lead to price wars and reduced profit margins.',
 'Technology Dependence: Reliance on advanced technology and IoT systems means that any technical failures or cybersecurity issues could disrupt operations and impact productivity.',
 'Consumer Awareness: While interest in sustainable practices is growing, there may still be a lack of awareness or understanding among consumers about the benefits of vertical farming, which could affect market penetration.']

In [31]:
logs

[AIMessage(content="### Business Idea: Quantum Wealth Management Platform (QWMP)\n\n#### Overview:\nThe Quantum Wealth Management Platform (QWMP) is a cutting-edge financial services solution that leverages quantum computing technology to provide advanced portfolio optimization and risk management for individual and institutional investors. By utilizing quantum algorithms, QWMP aims to outperform traditional portfolio management strategies, offering clients superior returns and enhanced risk-adjusted performance.\n\n#### Key Features:\n\n1. **Quantum Portfolio Optimization**:\n   - Utilize quantum algorithms to analyze vast datasets and identify optimal asset allocations that traditional methods may overlook.\n   - Provide real-time adjustments to portfolios based on market conditions, risk tolerance, and investment goals.\n\n2. **Risk Assessment and Management**:\n   - Implement quantum simulations to model complex financial scenarios and assess potential risks more accurately.\n   - 

## 7. Experiment

Now that you understood how it works, experiment with new things.
- Improve memory
- Explore the Runnables
- Add More Complexity